In [1]:
import numpy as np
import pickle
from tqdm import tqdm, trange
from numba import jit, vectorize

In [2]:
with open('vols.pickle', 'rb') as handle:
    f_ints = pickle.load(handle)

f_ints_np - уникальные пары объемов из f_ints, отсортированные в лексикографическом порядке

In [3]:
f_ints_np = np.array(list(f_ints.itertuples(index=False, name=None)))
f_ints_np = np.unique(f_ints_np, axis=0)
f_ints_np = f_ints_np[np.lexsort((f_ints_np[:, 1], f_ints_np[:, 0]))]

Далее непосредственно получаем матрицу H - которая есть на самом деле матрица f(i, j). Здесь в качестве i, j я беру уникальные значения объемов bid и ask после увеличения цены, причем, чтобы убрать выбросы, беру значения до 0.95 квантиля
x_edges и y_edges это есть отсортированные вектора значений i и j.

In [4]:
x = f_ints['bid_vol_0']
y = f_ints['ask_vol_0']
x_edges = np.sort(x[x < x.quantile(0.95)].unique())
y_edges = np.sort(y[y < y.quantile(0.95)].unique())
H, xedges, yedges = np.histogram2d(x, y, bins=[x_edges, y_edges],
                                   density=True, range=[[1, x.quantile(0.95)], [1, y.quantile(0.95)]])

Функция для поиска пары (n, p) в лексикографически отсортированном массиве

In [5]:
def search(n, p, pairs_sorted):
    i = np.searchsorted(pairs_sorted[:, 0], n)
    if i != len(pairs_sorted) and pairs_sorted[i, 0] == n:
        while i != len(pairs_sorted) and pairs_sorted[i, 0] == n and pairs_sorted[i, 1] <= p:
            if pairs_sorted[i, 1] == p:
                return i
            i += 1
        return -1
    else:
        return -1

In [6]:
@vectorize
def integrand(t, n, p):
    if t == 0:
        return 2*n
    else:
        return  ((2 - np.cos(t) - np.sqrt((2 - np.cos(t))**2 - 1))**p) * np.sin(n*t) * np.cos(t/2) / np.sin(t/2)

@jit
def p1_up(n_1, p_1, k=5):
    int_interval = np.linspace(0, np.pi, num=10**k+1, endpoint=True)
    return np.trapz(integrand(int_interval, n_1, p_1), x=int_interval) / np.pi

get_p1_up_arr - сюда подаем массив пар значений объемов (в нашем случае f_ints_np)

In [7]:
@jit(cache=True, parallel=True)
def get_p1_up_arr(pairs):
    matrix = np.empty(len(pairs), dtype=np.float64)
    i = 0
    #matrix = np.array([])
    for a_i, b_i in tqdm(pairs):
        j = 5
        while j <= 10:
            p = p1_up(a_i, b_i, j)
            if p <= 1:
                matrix[i] = p
                break
            else:
                j += 1
                matrix[i] = 0
        i += 1
    return matrix

Посчитаем p1_up только для тех пар объемов, которые были в данных

In [8]:
p1_up_arr = get_p1_up_arr(f_ints_np)

C:\Users\alpat\AppData\Local\Temp\ipykernel_8856\4202418940.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_p1_up_arr" failed type inference due to: Untyped global name 'tqdm': Cannot determine Numba type of <class 'type'>

File "C:\Users\alpat\AppData\Local\Temp\ipykernel_8856\4202418940.py", line 6:
def get_p1_up_arr(pairs):
    <source elided>
    #matrix = np.array([])
    for a_i, b_i in tqdm(pairs):
    ^

  @jit(cache=True, parallel=True)
C:\Users\alpat\AppData\Local\Temp\ipykernel_8856\4202418940.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "get_p1_up_arr" failed type inference due to: Untyped global name 'tqdm': Cannot determine Numba type of <class 'type'>

File "C:\Users\alpat\AppData\Local\Temp\ipykernel_8856\4202418940.py", line 6:
def get_p1_up_arr(pairs):
    <source elided>
    #matrix = np.array([])
    for a_i, b_i in tqdm(pairs):
    ^

  @ji

get_p1_up_matrix - возвращает матрицу интегралов, такого же размера, что и H. На местах, соответствующих парам объемов из массива pairs (в нашем случае f_ints_np), стоят соответствующие интегралы из массива integrals (в нашем случае p1_up_arr), на остальных местах стоят 0.

In [9]:
def get_p1_up_matrix(integrals, pairs, a_arr, b_arr):
    matrix = np.zeros((len(a_arr), len(b_arr)), dtype=np.float64)
    for i in trange(len(a_arr)):
        a_i = a_arr[i]
        for j, b_j in enumerate(b_arr):
            num = search(a_i, b_j, pairs)
            if num != -1:
                matrix[i][j] = integrals[num]
    return matrix

Здесь мы подаём массивы x_edges и y_edges, не включая последние элементы, чтобы матрица на выходе имела такие же размеры, что и матрица Н

In [10]:
p1_up_matrix = get_p1_up_matrix(p1_up_arr, f_ints_np, x_edges[:-1], y_edges[:-1])

100%|██████████| 6438/6438 [01:22<00:00, 78.41it/s] 


In [11]:
with open('p1_ups.npy', 'wb') as file:
    np.save(file, p1_up_matrix)

In [12]:
p_cont = np.sum(np.multiply(H, p1_up_matrix))
p_cont

0.6714336072327123